In [7]:
# contig assembler from sequence data.  Assembles reads with 20 nt overlaps with up to 1 mismatch

passnum = 1
import random
from google.colab import drive
drive.mount('/content/gdrive')
filename = '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/Sample.Genome.Reads.' +str(passnum)+ '.txt'
contigname = '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/Sample.Genome.Contig.' +str(passnum)+ '.txt'
passname = '/content/gdrive/Shared drives/2.Python/Genomics/Data Files/Sample.Genome.Reads.' +str(passnum +1)+ '.txt'

with open(filename, 'r') as f:
  data = str(f.read())
  #print(repr(data)[:200])
  #print(repr(data)[-200:])
f.close()

reads = data.split('\n')
reads.pop()

#random.shuffle(reads)

allset = set()
for x in range(len(reads)):
  allset.add(str(x))
alllist = list(allset)

# to process raw sequence from a genbank file, use the routine below

# reverses sequences
def rev(string1):  
  string2 =''
  for x in string1:
    if   x == 'c': string2 += 'g'
    elif x == 'g': string2 += 'c'
    elif x == 't': string2 += 'a'
    elif x == 'a': string2 += 't'
  string3 = string2[::-1]  
  return(string3)

# prepares contig for comparison
def contigends(string1):
  left = string1[: window]
  right = string1[-window:]
  return(left, right)

def match(string1, string2):
  sum =0
  for x in range(len(string1)):
    if string1[x] == string2[x]:
      sum +=1
  ratio = sum / len(string1)
  return(ratio)

window = 20
contig = reads[0]
hitset = set()
hitset.add('0')
wlist =[]
mismatchlimit = 0.95
for v in range(round(len(reads) **2 / 2 )):  # reapeated cycles through reads, adding overlpas to the contig
  w = v % len(reads)
  if str(w) in hitset: 
    continue
  elif str(w) not in hitset:
    left, right = contigends(contig)
    forseq = reads[w]
    revseq = rev(reads[w])
    for x in range(len(forseq) - window):
      forquery = forseq[x: x + window]
      forrightratio = match(forquery, right)
      forleftratio = match(forquery, left)
      revquery = revseq[x: x + window]
      revrightratio = match(revquery, right)
      revleftratio = match(revquery, left)
      if forrightratio >= mismatchlimit:
        contig += forseq[x + window:]
        hitset.add(str(w))
      elif forleftratio >= mismatchlimit:
        contig = forseq[:x] + contig
        hitset.add(str(w))
      elif revrightratio >= mismatchlimit:
        contig += revseq[x + window:]
        hitset.add(str(w))
      elif revleftratio >= mismatchlimit:
        contig = revseq[:x] + contig
        hitset.add(str(w))
      else: pass

print(contig)
total =0
for x in range(len(reads)):
  total += len(reads[x])
print('Length of the contig =', len(contig), ', Combined length of the reads =', total)
print('Total number of reads =', len(reads), ', Number of reads in the contig =', len(hitset))

hitlist = list(hitset)
for x in range(len(hitlist)):
  hitlist[x] = int(hitlist[x])

hitlist.sort() 
print('Reads in the contig:', hitlist)

missinglist =[]  
for x in range(len(alllist)):
  if alllist[x] not in hitset:
    missinglist.append(int(alllist[x]))

missinglist.sort()
print('Reads not in the contig:', missinglist)
if len(missinglist) > 0:
  outlist =[]
  for x in range(len(missinglist)):
    outlist.append(reads[int(missinglist[x])])

with open(passname, 'w') as g:
  for x in range(len(outlist)):
    g.write(outlist[x] + '\n')
g.close()

with open(contigname, 'w') as h:
  h.write(contig)
h.close()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
gcatattcgtacagttttttgccctgatatttcagtgctgaatacatcgaagggatctgttcgatatcgccacggaaagtatccagtgccgctgccagctgctctgcgctaaaggttaccggacgttcttcaacgatctgtccgtcggcatcagaagtatcggtacgctgtccaagacgcgcaatgacccgatagcgtttgtcggagtccagcagatactgggaaaacttcgtcgcttccccgaggcaaatcggcaacatgccggtcgccagcgggtccagcgcaccggtatgcccggcacggttggcgttatatatacgtttcactttttgcagcgcatcgttgctggacataccctgaggtttatccagcaacaaaacgccgttaatgtcgcgaccgcgacgacgaggacgactcattagtcctccttgctgtcgtccgggttaacacgacgttcttcgtcatgtttgaccacgctggtcaccaggtttgacatgcgcatcccttcaaccagagagttgtcgtagaagaaggtcagttccggcacgatacgcaggcgcatcgctttccccagcaggctgcggatgaaaccagaagcttcttgcaacgctttgatgcccgctttaaccgcgtcttcatctttgtcgttgaggaacgtcacatatactttggcatacgccaggtcgcgagacatttcgacaccggaaacggtggtcatcatgcccaggcgaggatctttaatttcacgctgcaggatgagagcgatctctttttgcatttcctgcgctacgcgctgcgggcgaccaaattctttcgccataataaattctcctgacaaaaaaggggctgttagcccctttttaaaatta

In [2]:
# routine for reversing a sequence 

string1 =  '1158121 tctcgcacgg tctgatcgacttcatcgttc tgtctggtaa cagcagcaaa ctgtggctgt tcccgatcgt cggtatcggt'

string2 =''
for x in string1:
  if x == 'c': string2 += 'g'
  elif x == 'g': string2 += 'c'
  elif x == 't': string2 += 'a'
  elif x == 'a': string2 += 't'

string3 = string2[::-1]
print(string3)



accgataccgacgatcgggaacagccacagtttgctgctgttaccagacagaacgatgaagtcgatcagaccgtgcgaga
